# data.csv

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/Basil1361/March-2026-Workshop-Resources/master/Resources/data.csv"

table = pd.read_csv(url)
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Duration  32 non-null     int64  
 1   Date      31 non-null     object 
 2   Pulse     32 non-null     int64  
 3   Maxpulse  32 non-null     int64  
 4   Calories  30 non-null     float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.4+ KB


In [4]:
# 1. count the number of missing values per column
table.isna().sum()

Duration    0
Date        1
Pulse       0
Maxpulse    0
Calories    2
dtype: int64

In [5]:
# 2. count the number of records where pulse > maxpulse
(table["Pulse"] > table["Maxpulse"]).sum()

1

In [6]:
# 3. remove missing/invalid values
table_clean = table[
    (table["Pulse"] <= table["Maxpulse"]) &
    (table["Duration"] <= 240)      
].reset_index(drop=True)

table_clean = table_clean.dropna()
table_clean["Date"] = pd.to_datetime(table_clean["Date"], errors="coerce", format="mixed")

table_clean.head(5)

,Duration,Date,Pulse,Maxpulse,Calories
0,60,2020-12-01,110,130,409.1
1,60,2020-12-02,117,145,479.0
2,60,2020-12-03,103,135,340.0
3,45,2020-12-04,109,175,282.4
4,45,2020-12-05,117,148,406.0


In [7]:
# 4. calculate the average pulse rate and average duration of all workouts
avg_pulse = table_clean["Pulse"].mean()
avg_duration = table_clean["Duration"].mean()

print(f"Average pulse is: {avg_pulse:.2f}")
print(f"Average duration is: {avg_duration:.2f}")

Average pulse is: 103.15
Average duration is: 56.67


In [8]:
# 5. create new feature (intensity)
table_clean["Intensity"] = "Medium"

table_clean.loc[table_clean["Pulse"] <= 100, "Intensity"] = "Low"
table_clean.loc[table_clean["Pulse"] >= 120, "Intensity"] = "High"

table_clean["Intensity"].value_counts()

Intensity
Medium    16
Low       11
Name: count, dtype: int64

# police.csv

In [9]:
import pandas as pd
url = "https://raw.githubusercontent.com/Basil1361/March-2026-Workshop-Resources/refs/heads/master/Resources/police.csv"

police = pd.read_csv(url)
police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   stop_date           91741 non-null  object 
 1   stop_time           91741 non-null  object 
 2   county_name         0 non-null      float64
 3   driver_gender       86406 non-null  object 
 4   driver_age_raw      86414 non-null  float64
 5   driver_age          86120 non-null  float64
 6   driver_race         86408 non-null  object 
 7   violation_raw       86408 non-null  object 
 8   violation           86408 non-null  object 
 9   search_conducted    91741 non-null  bool   
 10  search_type         3196 non-null   object 
 11  stop_outcome        86408 non-null  object 
 12  is_arrested         86408 non-null  object 
 13  stop_duration       86408 non-null  object 
 14  drugs_related_stop  91741 non-null  bool   
dtypes: bool(2), float64(3), object(10)
memory usage: 9.3+

In [10]:
# 1. count missing values
police.isna().sum()

stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5335
driver_age_raw         5327
driver_age             5621
driver_race            5333
violation_raw          5333
violation              5333
search_conducted          0
search_type           88545
stop_outcome           5333
is_arrested            5333
stop_duration          5333
drugs_related_stop        0
dtype: int64

In [11]:
# 2. count for types of violations
police_new = police.drop(columns=['county_name', 'search_type'])
police_new['violation'] = police_new['violation'].astype(str)
police_new['violation'].value_counts()

violation
Speeding               48463
Moving violation       16224
Equipment              11020
nan                     5333
Other                   4317
Registration/plates     3432
Seat belt               2952
Name: count, dtype: int64

In [ ]:
# 3. violation type counts by gender
police_new_cleaned = police_new.dropna()
police_new_cleaned.groupby('violation')['driver_gender'].value_counts()

violation            driver_gender
Equipment            M                 8523
                     F                 2484
Moving violation     M                12969
                     F                 3195
Other                M                 3519
                     F                  685
Registration/plates  M                 2415
                     F                 1012
Seat belt            M                 2317
                     F                  635
Speeding             M                32906
                     F                15453
Name: count, dtype: int64

In [13]:
# 4. create age categories
police_new_cleaned["age_category"] = "Middle-Aged"

police_new_cleaned.loc[police_new_cleaned["driver_age"] >= 65.0, "age_category"] = "Senior Citizen"
police_new_cleaned.loc[police_new_cleaned["driver_age"] < 18.0, "age_category"] = "Underage"
police_new_cleaned.loc[(18.0 < police_new_cleaned["driver_age"]) & (police_new_cleaned["driver_age"] <= 35.0), "age_category"] = "Young Adult"

police_new_cleaned["age_category"].value_counts()

C:\Users\User\AppData\Local\Temp\ipykernel_10008\2512518327.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_new_cleaned["age_category"] = "Middle-Aged"


age_category
Young Adult       50688
Middle-Aged       33211
Senior Citizen     1579
Underage            635
Name: count, dtype: int64

In [15]:
# 5. median driver age for both genders
police_new_cleaned.groupby("driver_gender")["driver_age"].median()

driver_gender
F    29.0
M    31.0
Name: driver_age, dtype: float64

In [ ]:
# 6. is_speeding category with boolean values
police_new_cleaned["is_speeding"] = (
    police_new_cleaned["violation"]
        .eq("Speeding")
        .replace({True: "Speeding", False: "Not Speeding"})
)

police_new_cleaned["is_speeding"].value_counts()

In [ ]:
# 7. convert to datetime and display year with the most speeding violations
import pandas as pd
police_new_cleaned["stop_date"] = pd.to_datetime(police_new_cleaned["stop_date"])

(
    police_new_cleaned
        .loc[police_new_cleaned["violation"] == "Speeding", "stop_date"]
        .dt.year
        .value_counts()
        .idxmax()
)

C:\Users\User\AppData\Local\Temp\ipykernel_10008\3137969831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_new_cleaned["stop_date"] = pd.to_datetime(police_new_cleaned["stop_date"])


2006

In [ ]:
# 8. violation rank according the frequency by gender
freq = (
    police_new_cleaned
        .groupby(["driver_gender", "violation"])
        .size()
        .reset_index(name="count")
)

freq["violation_rank"] = (
    freq.groupby("driver_gender")["count"]
        .rank(method="dense", ascending=False)
        .astype(int)
)


,driver_gender,violation,count,violation_rank
0,F,Equipment,2484,3
1,F,Moving violation,3195,2
2,F,Other,685,5
3,F,Registration/plates,1012,4
4,F,Seat belt,635,6
5,F,Speeding,15453,1
6,M,Equipment,8523,3
7,M,Moving violation,12969,2
8,M,Other,3519,4
9,M,Registration/plates,2415,5
